In [2]:
import pandas as pd
import scanpy as sp
from tqdm import tqdm

In [3]:
outpath = "output/make_overview_table/"

In [4]:
samples = pd.read_csv("/oak/stanford/groups/horence/JuliaO/visium_analysis/notebooks/output/make_samplesheet/spatial.csv",index_col = 0)
scores = pd.read_csv("output/make_samplesheet/scores.csv",index_col=0)

In [5]:
sub_scores = ["SpliZ", "ReadZS", "ge", "ReadZS_ge"]
out = {"dataname" : [], "num_spots" : [], "med_reads_per_spot" : []}
for score in sub_scores:
  out["{}_med_per_spot".format(score)] = []
  
# datanames = ['V1_Mouse_Brain_Sagittal_Posterior',
#        'V1_Mouse_Brain_Sagittal_Posterior_Section_2',
#        'V1_Mouse_Brain_Sagittal_Anterior',
#        'V1_Mouse_Brain_Sagittal_Anterior_Section_2', 'V1_Mouse_Kidney',
#        'Visium_FFPE_Human_Breast_Cancer', 'Visium_FFPE_Human_Normal_Prostate',
#        'Visium_FFPE_Human_Prostate_Acinar_Cell_Carcinoma',
#        'Visium_FFPE_Human_Prostate_Cancer', 'Visium_FFPE_Human_Prostate_IF',
#        'Visium_FFPE_Mouse_Brain', 'Visium_FFPE_Mouse_Brain_IF',
#        'Visium_FFPE_Mouse_Kidney',  'p20190_s003_3_BrainMetastasis',
#        'p20190_s004_4_BrainMetastasis', 'p20218_s001_L1', 'p20218_s002_L2',
#        'p20218_s003_L3', 'p20218_s004_L4']

datanames = ["V1_Mouse_Brain_Sagittal_Posterior","V1_Mouse_Brain_Sagittal_Posterior_Section_2",
             "V1_Mouse_Brain_Sagittal_Anterior","V1_Mouse_Brain_Sagittal_Anterior_Section_2",
             "V1_Mouse_Kidney","p20218_s002_L2","p20218_s001_L1"]

for dataname, row in tqdm(samples[samples["method"] == "visium"].iterrows()):

  if dataname in datanames:
    out["dataname"].append(dataname)
    meta = pd.read_csv(row["metadata"],sep="\t")
    out["num_spots"].append(meta["in_tissue"].sum())
    try:
      data = sp.read_mtx(row["ge_mat"] + "matrix.mtx.gz")
      out["med_reads_per_spot"].append(data.to_df().sum(axis=0).median())
    except:
      out["med_reads_per_spot"].append(np.nan)
    for score, srow in scores.iterrows():
      if score in sub_scores:

        try:
          df = pd.read_csv(row[srow["valname"]],sep="\t",usecols = [srow["cellid"],srow["genecol"]])
          out["{}_med_per_spot".format(score)].append(df.groupby(srow["cellid"])[srow["genecol"]].nunique().median())

        except:
          out["{}_med_per_spot".format(score)].append(np.nan)
out = pd.DataFrame.from_dict(out)
out

20it [07:18, 21.91s/it]


,dataname,num_spots,med_reads_per_spot,SpliZ_med_per_spot,ReadZS_med_per_spot,ge_med_per_spot,ReadZS_ge_med_per_spot
0,V1_Mouse_Brain_Sagittal_Posterior,3355,15783.0,203.0,3178.0,6628.0,2812.0
1,V1_Mouse_Brain_Sagittal_Posterior_Section_2,3289,14262.0,163.0,2905.0,6196.0,2640.0
2,V1_Mouse_Brain_Sagittal_Anterior,2695,25888.0,182.0,3290.0,7062.0,4061.0
3,V1_Mouse_Brain_Sagittal_Anterior_Section_2,2825,20145.0,222.5,2954.0,6273.0,3173.0
4,V1_Mouse_Kidney,1436,38308.0,56.0,253.5,3516.0,1212.0
5,p20218_s001_L1,4392,3656.5,274.0,812.0,1863.0,307.0
6,p20218_s002_L2,2215,844.0,199.0,475.0,176.0,76.0


In [7]:
data.to_df().sum(axis=0).median()

125.0

In [8]:
out = pd.DataFrame.from_dict(out)

In [9]:
out.to_csv("{}table.tsv".format(outpath),sep="\t",index=False)

In [5]:
out = pd.read_csv("{}table.tsv".format(outpath),sep="\t")

In [10]:
out.dropna().to_csv("{}spatial_table1.tsv".format(outpath),sep="\t",index=False)

In [21]:
meta["in_tissue"].sum()

1684

In [22]:
data.to_df().sum(axis=0).median()

125.0

In [5]:
for k, v in out.items():
  print(k,len(v))

dataname 4
num_spots 4
med_reads_per_spot 16
SpliZ_med_per_spot 4
ReadZS_med_per_spot 4
ge_med_per_spot 4
ReadZS_ge_med_per_spot 4


In [33]:
out = pd.DataFrame.from_dict(out)
out

,dataname,num_spots,SpliZ_med_per_spot,ReadZS_med_per_spot,ge_med_per_spot,ReadZS_ge_med_per_spot
0,V1_Mouse_Kidney,1438,26.0,253.5,3516.0,1212.0
